Count of First Sentence in comments

In [24]:
4. #Seperate out first sentence mark as duplicate or not if >2
def first_sentence (x):
    """Removes first Sentence"""
    z = str(x).split('.')
    if len(z) == 1:
        return None
    else:
        return z[0] 

df['first_sentence'] = df['comment_text'].apply(first_sentence)    
#count first sentences for multiples    
df['first_sentence_freq'] = df.groupby('first_sentence')['first_sentence'].transform('count')   

In [32]:
df.first_sentence.value_counts().head(5)

The FCC's Open Internet Rules (net neutrality rules) are extremely important to me                                                                                                               410727
I strongly oppose Chairman Pai&apos;s proposal to reverse net neutrality protections because a free and open internet is vital for our democracy, for our businesses, and for our daily lives    296941
The unprecedented regulatory power the Obama Administration imposed on the internet is smothering innovation, damaging the American economy and obstructing job creation                         176258
Cable and phone companies provide access to the internet                                                                                                                                          13862
I support Chairman Ajit Pai?s rollback of Obama?s internet takeover                                                                                                                                5555


In [26]:
# filer name frequency.. what counts as too high?
df['filers_freq'] = df.groupby('filers')['filers'].transform('count') 

In [31]:
df.filers.value_counts().head(5)

[{"name":"Net Neutrality"}]    1834
[{"name":"Margie Garman"}]     1293
[{"name":"Pat M"}]              898
[{"name":"net neutrality"}]     807
[{"name":"Michael"}]            502
Name: filers, dtype: int64

In [33]:
df.contact_email.value_counts().head(5)

fcctest@mailinator.com      174
zoeprice.moon@gmail.com      63
airagorncharda@yahoo.com     61
angelsggrandma@yahoo.com     46
starstrider@q.com            43
Name: contact_email, dtype: int64

#Wrangle Dataset for Logistic Regression

In [2]:
#imports
import pandas as pd

In [3]:
#Read in Data
df = pd.read_csv('FCC_US_commentstext_datefreq.csv',encoding = 'latin-1')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (5,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#Already have 400 test and train data marked with Fake/True Flag
#merge 4 datasets into 1

In [36]:
#read in test and training data - graded data
df1 = pd.read_csv('FCC_df_zip5_end.csv')
df2 = pd.read_csv('FCC_df_zip5.csv')
df3 = pd.read_csv('FCC_df_zip5_mid.csv')
df4 = pd.read_csv('FCC_df_badzip.csv')

data = pd.concat([df1,df2,df3,df4])

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 0 to 99
Data columns (total 15 columns):
id                            400 non-null object
date_submission               400 non-null object
contact_email                 389 non-null object
confirmation_number           400 non-null float64
text_data                     397 non-null object
index                         400 non-null object
filers                        400 non-null object
addressentity                 400 non-null object
internationaladdressentity    400 non-null object
address_line_1                400 non-null object
city                          400 non-null object
state                         400 non-null object
zip_code                      400 non-null object
international_address_text    0 non-null float64
fake                          400 non-null int64
dtypes: float64(2), int64(1), object(12)
memory usage: 50.0+ KB


In [106]:
#explore concatonated graded data
data.head(2)

,id,date_submission,contact_email,confirmation_number,comment_text,index,filers,addressentity,internationaladdressentity,address_line_1,city,state,zip_code,international_address_text,fake,date_test,intl_fake_zip
0,59f88aa650b733300fe2da17,2017-05-25T16:25:26.831Z,bobbywego@gmail.com,2.017050e+15,I am deeply concerned over the Notice of Propo...,filings.2017.5,"[{""name"":""Robert Wengronowitz""}]","{'city': 'Davenport', 'address_line_1': '2514 ...",{'addresstext': ''},2514 E. 18th St.,Davenport,IA,52803,NaN,0,2017-05-25 16:25:26,0
1,59f88aa650b733300fe2da18,2017-05-25T16:25:42.688Z,ganriel.thomason@gmail.com,2.017050e+16,The FCC should safeguard Internet freedom by k...,filings.2017.5,"[{""name"":""Gabriel Thomason""}]","{'city': 'Warner Robins', 'address_line_1': '1...",{'addresstext': ''},123 SRingwood Dr.,Warner Robins,GA,31088,NaN,0,2017-05-25 16:25:42,0


In [96]:
data=data.rename(columns = {'text_data':'comment_text'})

In [105]:
#add intl fake zip column to match full dataset (where zip code was extracted from suspicious address)
data['intl_fake_zip']=0

In [107]:
#add model column 
data['model']=1

In [119]:
data.columns

Index(['id', 'date_submission', 'contact_email', 'confirmation_number',
       'comment_text', 'index', 'filers', 'addressentity',
       'internationaladdressentity', 'address_line_1', 'city', 'state',
       'zip_code', 'international_address_text', 'fake', 'date_test',
       'intl_fake_zip', 'model'],
      dtype='object')

In [139]:
model_df = data[['id', 'date_submission', 'contact_email',  \
       'comment_text', 'index', 'filers', \
       'internationaladdressentity', 'fake', 'date_test',\
       'intl_fake_zip', 'model']]

In [140]:
model_df.head(2)

,id,date_submission,contact_email,comment_text,index,filers,internationaladdressentity,fake,date_test,intl_fake_zip,model
0,59f88aa650b733300fe2da17,2017-05-25T16:25:26.831Z,bobbywego@gmail.com,I am deeply concerned over the Notice of Propo...,filings.2017.5,"[{""name"":""Robert Wengronowitz""}]",{'addresstext': ''},0,2017-05-25 16:25:26,0,1
1,59f88aa650b733300fe2da18,2017-05-25T16:25:42.688Z,ganriel.thomason@gmail.com,The FCC should safeguard Internet freedom by k...,filings.2017.5,"[{""name"":""Gabriel Thomason""}]",{'addresstext': ''},0,2017-05-25 16:25:42,0,1


--
make some edits to full dataset

In [117]:
df['model']=0

In [94]:
# to merge both data sets- remove 'State_Abbreviation Column from full data
df_full.drop('State_Abbreviation',axis=1,inplace=True)

In [111]:
df.drop('renamed_text_data',axis=1,inplace=True)

In [112]:
df.drop('text_data',axis=1,inplace=True)

In [91]:
data['date_test']= pd.to_datetime(data['date_submission'])
data['date_test']= data['date_test'].values.astype('datetime64[s]')

In [83]:
#select a cell's contents
#df[df.id=='59f88aa650b733300fe2da18'].text_data.item()

In [108]:
#select a random sample from the dataset
#df.sample(n=400)

In [115]:
df.head(2)

,State_Abbreviation,id,date_submission,contact_email,index,internationaladdressentity,filers,intl_fake_zip,date_test,freq_time,comment_text,email_freq,repeated_email,email_upper,repeated_time,first_sentence,first_sentence_freq,filers_freq,filers_name_freq,flag_intl_fake_zip
0,IA,59f8b2bb50b733300f069dce,2017-07-11T15:53:55.588Z,LINDAKUNZE88@GMAIL.COM,filings.2017.7,NaN,"[{""name"":""Linda Kunze""}]",NaN,2017-07-11 15:53:55,12,The unprecedented regulatory power the Obama A...,1.0,0,1,1,The unprecedented regulatory power the Obama A...,176258.0,1,1,0
1,RI,59f8b2bb50b733300f069dd1,2017-07-11T15:53:55.750Z,TD3ONE@GMAIL.COM,filings.2017.7,NaN,"[{""name"":""Thomas Daniels""}]",NaN,2017-07-11 15:53:55,12,The unprecedented regulatory power the Obama A...,1.0,0,1,1,The unprecedented regulatory power the Obama A...,176258.0,2,2,0


In [133]:
df_intl_fake = df[df['intl_fake_zip']==1]

In [134]:
df_intl_fake = df_intl_fake[:100]

In [136]:
df_intl_fake['fake'] = 1

In [138]:
model_df2 = df_intl_fake[['id', 'date_submission', 'contact_email',  \
       'comment_text', 'index', 'filers', \
       'internationaladdressentity', 'fake', 'date_test',\
       'intl_fake_zip', 'model']]

In [143]:
model_df2.head()

,id,date_submission,contact_email,comment_text,index,filers,internationaladdressentity,fake,date_test,intl_fake_zip,model
2306602,59f8b2bb50b733300f069dc9,2017-07-11T15:53:55.361Z,YaroslavKalinina@pornhub.com,NaN,filings.2017.7,"[{""name"":""Yaroslav Kalinina""}]","{""addresstext"":""3575 Eagle Lane\n55792,Virgini...",1,2017-07-11 15:53:55,1.0,1
2306603,59f8b2bb50b733300f069dca,2017-07-11T15:53:55.390Z,GalinaMedvedeva@pornhub.com,NaN,filings.2017.7,"[{""name"":""Galina Medvedeva""}]","{""addresstext"":""2245 Lodgeville Road\n55415,Mi...",1,2017-07-11 15:53:55,1.0,1
2306604,59f8b2bb50b733300f069dcb,2017-07-11T15:53:55.449Z,PhilemonButusov@pornhub.com,NaN,filings.2017.7,"[{""name"":""Philemon Butusov""}]","{""addresstext"":""4702 Ashmor Drive\n55802,Dulut...",1,2017-07-11 15:53:55,1.0,1
2306605,59f8b2bb50b733300f069dcc,2017-07-11T15:53:55.473Z,BarbaraUlyanova@pornhub.com,NaN,filings.2017.7,"[{""name"":""Barbara Ulyanova""}]","{""addresstext"":""2323 Lodgeville Road\n55406,Mi...",1,2017-07-11 15:53:55,1.0,1
2306606,59f8b2bb50b733300f069dcd,2017-07-11T15:53:55.512Z,MatildaZhdanova@pornhub.com,NaN,filings.2017.7,"[{""name"":""Matilda Zhdanova""}]","{""addresstext"":""2135 Lena Lane\n39401,Hattiesb...",1,2017-07-11 15:53:55,1.0,1


In [142]:
model_df2['model'] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [144]:
model_df1 = pd.concat([model_df,model_df2])

In [145]:
model_df1.head()

,id,date_submission,contact_email,comment_text,index,filers,internationaladdressentity,fake,date_test,intl_fake_zip,model
0,59f88aa650b733300fe2da17,2017-05-25T16:25:26.831Z,bobbywego@gmail.com,I am deeply concerned over the Notice of Propo...,filings.2017.5,"[{""name"":""Robert Wengronowitz""}]",{'addresstext': ''},0,2017-05-25 16:25:26,0.0,1
1,59f88aa650b733300fe2da18,2017-05-25T16:25:42.688Z,ganriel.thomason@gmail.com,The FCC should safeguard Internet freedom by k...,filings.2017.5,"[{""name"":""Gabriel Thomason""}]",{'addresstext': ''},0,2017-05-25 16:25:42,0.0,1
2,59f88aa650b733300fe2da19,2017-05-25T16:26:32.472Z,kyliepborges@gmail.com,"I support strong net neutrality, backed by Tit...",filings.2017.5,"[{""name"":""Kylie Borges""}]",{'addresstext': ''},0,2017-05-25 16:26:32,0.0,1
3,59f88aa650b733300fe2da1b,2017-05-25T16:28:36.684Z,NaN,Preserve net neutrality and title II. Don't le...,filings.2017.5,"[{""name"":""Joe D.""}]",{'addresstext': ''},0,2017-05-25 16:28:36,0.0,1
4,59f88aa650b733300fe2da1c,2017-05-25T16:30:32.053Z,sarahaarnold@outlook.com,I specifically support strong net neutrality t...,filings.2017.5,"[{""name"":""Sarah Arnold""}]",{'addresstext': ''},0,2017-05-25 16:30:32,0.0,1


In [147]:
df_full = df[['id', 'date_submission', 'contact_email',  \
       'comment_text', 'index', 'filers', \
       'internationaladdressentity',  'date_test',\
       'intl_fake_zip', 'model']]

In [148]:
df_combined = pd.concat([df_full,model_df1])

In [149]:
df_combined.head()

,comment_text,contact_email,date_submission,date_test,fake,filers,id,index,internationaladdressentity,intl_fake_zip,model
0,The unprecedented regulatory power the Obama A...,LINDAKUNZE88@GMAIL.COM,2017-07-11T15:53:55.588Z,2017-07-11 15:53:55,NaN,"[{""name"":""Linda Kunze""}]",59f8b2bb50b733300f069dce,filings.2017.7,NaN,NaN,0
1,The unprecedented regulatory power the Obama A...,TD3ONE@GMAIL.COM,2017-07-11T15:53:55.750Z,2017-07-11 15:53:55,NaN,"[{""name"":""Thomas Daniels""}]",59f8b2bb50b733300f069dd1,filings.2017.7,NaN,NaN,0
2,The unprecedented regulatory power the Obama A...,TJ-VANTASSEL@HOTMAIL.COM,2017-07-11T15:54:43.314Z,2017-07-11 15:54:43,NaN,"[{""name"":""Jean Vantassel""}]",59f8b2bb50b733300f06a0cf,filings.2017.7,NaN,NaN,0
3,The unprecedented regulatory power the Obama A...,NIKKISHERLEY@GMAIL.COM,2017-07-11T15:54:44.053Z,2017-07-11 15:54:44,NaN,"[{""name"":""Nicole Sherley""}]",59f8b2bb50b733300f06a0df,filings.2017.7,NaN,NaN,0
4,The unprecedented regulatory power the Obama A...,PAULAMUTTER61@GMAIL.COM,2017-07-11T15:54:44.444Z,2017-07-11 15:54:44,NaN,"[{""name"":""Paula Mutter""}]",59f8b2bb50b733300f06a0e7,filings.2017.7,NaN,NaN,0


In [150]:
#remove microsecond date submission:
df_combined.drop('date_submission',axis=1,inplace=True)

In [151]:
#remove index(filing) date submission:
df_combined.drop(['index','internationaladdressentity'],axis=1,inplace=True)

In [160]:
1. #Mark email as all capitalized or not
def email_upper (x):
    if x is None:
        return 0
    elif str(x).isupper():
        return 1
    else:
        return 0
    
#mark emails as being all uppercase or not    
df_combined['flag_email_upper'] = df_combined['contact_email'].apply(email_upper)   

In [161]:
2. #Mark email as repeated or not if >2
def multiples_2 (x):
    if x > 2:
        return 1
    else:
        return 0
    
#count emails for multiple submissions    
df_combined['email_freq'] = df_combined.groupby('contact_email')['contact_email'].transform('count')   

In [162]:
df_combined['flag_repeated_email'] = df_combined['email_freq'].apply(multiples_2)

In [ ]:
#do time calculation on data

In [163]:
#Convert data of test set to seconds and count freq
#df['date_test']= pd.to_datetime(df['date_submission'])
#df['date_test']= df['date_test'].values.astype('datetime64[s]')
df_combined['freq_time'] = df_combined.groupby('date_test')['date_test'].transform('count')
df_combined['freq_time'] = pd.to_numeric(df_combined['freq_time'])

In [164]:
3. #Mark time as repeated or not if >2
df_combined['flag_repeated_time'] = df_combined['freq_time'].apply(multiples_2)

In [165]:
def convert_0_1(x):
    if x is None:
        return 0
    elif x == 1:
        return 1
    else:
        return 0


df_combined['flag_intl_fake_zip'] = df_combined['intl_fake_zip'].apply(convert_0_1)


In [166]:
df_combined.head(2)

,comment_text,contact_email,date_test,fake,filers,id,intl_fake_zip,model,flag_email_upper,email_freq,flag_repeated_email,freq_time,flag_repeated_time,flag_intl_fake_zip
0,The unprecedented regulatory power the Obama A...,LINDAKUNZE88@GMAIL.COM,2017-07-11 15:53:55,NaN,"[{""name"":""Linda Kunze""}]",59f8b2bb50b733300f069dce,NaN,0,1,1.0,0,22,1,0
1,The unprecedented regulatory power the Obama A...,TD3ONE@GMAIL.COM,2017-07-11 15:53:55,NaN,"[{""name"":""Thomas Daniels""}]",59f8b2bb50b733300f069dd1,NaN,0,1,1.0,0,22,1,0


In [167]:
df_combined.columns

Index(['comment_text', 'contact_email', 'date_test', 'fake', 'filers', 'id',
       'intl_fake_zip', 'model', 'flag_email_upper', 'email_freq',
       'flag_repeated_email', 'freq_time', 'flag_repeated_time',
       'flag_intl_fake_zip'],
      dtype='object')

In [170]:
df_sample= df_combined[['id','flag_email_upper','flag_repeated_email','flag_repeated_time','flag_intl_fake_zip','model','fake']]

In [171]:
df_sample.head()

,id,flag_email_upper,flag_repeated_email,flag_repeated_time,flag_intl_fake_zip,model,fake
0,59f8b2bb50b733300f069dce,1,0,1,0,0,NaN
1,59f8b2bb50b733300f069dd1,1,0,1,0,0,NaN
2,59f8b2bb50b733300f06a0cf,1,0,1,0,0,NaN
3,59f8b2bb50b733300f06a0df,1,0,1,0,0,NaN
4,59f8b2bb50b733300f06a0e7,1,0,1,0,0,NaN


In [172]:
df_train_test = df_sample[df_sample['model']==1]

In [173]:
df_data = df_sample[df_sample['model']==0]

In [181]:
#500 records will be train/test
df_train_test.head()


,flag_email_upper,flag_repeated_email,flag_repeated_time,flag_intl_fake_zip,fake
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


In [175]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [179]:
df_train_test.drop(['model','id'],axis=1,inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [180]:
df_train_test.fake = df_train_test.fake.astype(int)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [182]:
X = df_train_test.drop('fake', axis=1) #remove survive from data
y = df_train_test['fake']#predict

In [183]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=101)

In [184]:
logmodel = LogisticRegression()

In [185]:
logmodel.fit(X_train,y_train) #shows default parameters

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [186]:
predictions = logmodel.predict(X_test)

In [187]:
from sklearn.metrics import classification_report

In [188]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       1.00      0.98      0.99        49
          1       1.00      1.00      1.00       201

avg / total       1.00      1.00      1.00       250



In [189]:
from sklearn.metrics import confusion_matrix

In [190]:
print(confusion_matrix(y_test,predictions))

[[ 48   1]
 [  0 201]]


In [196]:
predictions

array([0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [197]:
len(predictions)

250

In [205]:
# AUC score
from sklearn.metrics import roc_auc_score
# logmodel = LogisticRegression() # already done above
# X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.4, random_state=42)
# logmodel.fit(X_train, y_train)
y_pred_prob = logmodel.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_pred_prob)

0.99888313534368967

In [206]:
print("AUC: {}".format(roc_auc_score(y_test,y_pred_prob)))

AUC: 0.9988831353436897


In [207]:
# Cross validation scores
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(logmodel, X, y, cv=5,scoring='roc_auc')
print(cv_scores) 

[ 1.          0.97727273  0.94605184  0.99114774  1.        ]
